In [3]:
!pip install wget

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=1a3f9dd24b40dad1cf8568cc9fecc77684bc76d56b9ad07746a91f2e317d7262
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\8b\f1\7f\5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [6]:
import wget

In [11]:
# 이미지 파일 다운
wget.download('http://images.cocodataset.org/zips/val2017.zip') 

100% [......................................................................] 815585330 / 815585330

'val2017.zip'

In [63]:
# 어노테이션 다운
wget.download('http://images.cocodataset.org/annotations/annotations_trainval2017.zip')

100% [......................................................................] 252907541 / 252907541

'annotations_trainval2017.zip'

In [32]:
# 총 개수 확인
len(os.listdir('val2017/val2017'))

5000

In [ ]:
#파일이름 맞추기
for i in os.listdir('val2017/val2017'):
    os.rename(f'val2017/val2017/{i}', f'val2017/val2017/{i.lstrip("0")}')

In [72]:
# 어노테이션 불러와서 데이터 뽑아오기
import json
json_path = 'annotations_trainval2017/annotations/instances_val2017.json'

with open(json_path, "r") as f:
    json_data = json.load(f)

In [56]:
#파일이름
json_data['images'][0]['file_name']
# 높이
json_data['images'][0]['height']
# 넓이
json_data['images'][0]['width']
# 파일 아이디
json_data['images'][0]['id']

'000000397133.jpg'

In [307]:
#가로,세로,파일이름 가져오기
w1= []
h1= []
f1= []
for i in json_data['images']:
    if i['file_name'].lstrip("0") in real_file_list:
        w1.append(i['width'])
        h1.append(i['height'])
        f1.append(i['file_name'].lstrip("0"))
        

In [308]:
#파일 이름 jpg 만 빼고 가져오고싶어서 사용
ff1 = []
for i in f1:
    ff1.append(i.replace('.jpg',''))

In [309]:
#오름차순 정렬 후 인덱스 초기화 
# 타입을 int로 바꿈 
df_3 = pd.DataFrame({'width':w1,'height':h1,'image':ff1})
df_3.sort_values('image').reset_index(drop=True)
df_4 = df_3.astype('int64')

In [347]:
df_1

,image,class,bbox
0,1675,17,"[0.0, 14.01, 640.0, 293.56]"
1,1675,76,"[61.48, 367.82, 508.05, 105.71]"
2,4795,73,"[0.0, 192.0, 174.74, 282.61]"
3,4795,17,"[152.77, 113.55, 395.36, 364.39]"
4,4795,72,"[19.95, 2.49, 620.05, 349.09]"
...,...,...,...
1754,574810,17,"[64.52, 93.55, 215.05, 366.67]"
1755,575357,18,"[250.5, 158.39, 366.49, 199.85]"
1756,575357,34,"[39.52, 129.49, 102.3, 59.73]"
1757,579321,1,"[94.78, 0.95, 240.66, 211.01]"


In [348]:
df_4

,width,height,image
0,640,427,219578
1,640,371,555705
2,500,375,443303
3,640,480,25560
4,500,375,403817
...,...,...,...
344,501,640,360943
345,349,500,572388
346,527,640,377000
347,640,479,198641


In [349]:
df_total = pd.merge(df_1,df_4, on='image')

349

In [350]:
len(set(df_total['image']))

349

In [351]:
bboxes = []
for i,w,h in zip(df_total['bbox'],df_total['width'],df_total['height']):
    x1, y1, x2, y2 = i[0],i[1],i[2],i[3]
    image_width, image_height = w, h 
    
    x_center = (x1 + x2) / (2 * image_width)
    y_center = (y1 + y2) / (2 * image_height)
    yolo_width = x2 / image_width
    yolo_height = y2 / image_height
    yolo_bbox = [x_center, y_center, yolo_width, yolo_height]

    bboxes.append(yolo_bbox)


    # YOLO 형식의 바운딩 박스 좌표 출력
    


In [352]:
df_total['bbox'] = bboxes

In [353]:
df_total

,image,class,bbox,width,height
0,1675,17,"[0.5, 0.32038541666666664, 1.0, 0.611583333333...",640,480
1,1675,76,"[0.44494531249999997, 0.49326041666666665, 0.7...",640,480
2,4795,73,"[0.136515625, 0.4943854166666667, 0.27303125, ...",640,480
3,4795,17,"[0.4282265625, 0.49785416666666665, 0.61775, 0...",640,480
4,4795,72,"[0.5, 0.36622916666666666, 0.9688281249999999,...",640,480
...,...,...,...,...,...
1754,574810,17,"[0.3707824933687003, 0.46022, 0.57042440318302...",377,500
1755,575357,18,"[0.4820234375, 0.3954083885209713, 0.572640625...",640,453
1756,575357,34,"[0.11079687499999999, 0.20885209713024283, 0.1...",640,453
1757,579321,1,"[0.2620625, 0.21281124497991966, 0.37603125, 0...",640,498


In [364]:
last_df = df_total[(df_total['class']==17) | (df_total['class']==18)]

In [365]:
last_df.loc[last_df['class'] == 17, 'class'] = 0
last_df.loc[last_df['class'] == 18, 'class'] = 1
last_df

,image,class,bbox,width,height
0,1675,0,"[0.5, 0.32038541666666664, 1.0, 0.611583333333...",640,480
3,4795,0,"[0.4282265625, 0.49785416666666665, 0.61775, 0...",640,480
7,7386,1,"[0.18275833333333333, 0.408275, 0.064, 0.1137]",600,400
12,10363,0,"[0.36846875, 0.33792243767313024, 0.353328125,...",640,361
15,14007,0,"[0.3256875, 0.3502582159624413, 0.119734374999...",640,426
...,...,...,...,...,...
1745,572388,0,"[0.26545845272206303, 0.45294000000000006, 0.5...",349,500
1750,574315,0,"[0.3386875, 0.35516666666666663, 0.39334375, 0...",640,480
1754,574810,0,"[0.3707824933687003, 0.46022, 0.57042440318302...",377,500
1755,575357,1,"[0.4820234375, 0.3954083885209713, 0.572640625...",640,453


In [366]:
def create_yolo_label_txt(row):
    class_id = row['class']
    bbox = row['bbox']
    yolo_label = f"{class_id} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}"
    return yolo_label

for index, row in last_df.iterrows():
    txt_filename = str(row['image'])+'.txt'
    txt_filepath = './labels/'+txt_filename

    yolo_label = create_yolo_label_txt(row)

    with open(txt_filepath, 'w') as f:
        f.write(yolo_label)

print("YOLO label .txt files created in 'val_labels' folder.")

YOLO label .txt files created in 'val_labels' folder.


In [224]:
os.makedirs('labels',exist_ok=True)

In [99]:
#고양이 17 ,개 18번
#바운딩 박스, 파일이름, 클래스 이름 가져오기
bbox_list = []
image_list = []
cls_list = []
for a,b,c in zip(json_data['annotations'],json_data['annotations'],json_data['annotations']):
    bbox_list.append(a['bbox'])
    image_list.append(b['image_id'])
    cls_list.append(c['category_id'])

In [137]:
#고양이 17 ,개 18번
#바운딩 박스, 파일이름, 클래스 이름 가져오기

bbox_list = []
image_list = []
cls_list = []

for i in json_data['annotations']:
    if i['image_id'] in real_imageid_list:
        bbox_list.append(i['bbox'])
        image_list.append(i['image_id'])
        cls_list.append(i['category_id'])
        

In [367]:
import pandas as pd
df = pd.DataFrame({'image':image_list,'class':cls_list,'bbox':bbox_list})
df_1 = df.sort_values(by='image').reset_index(drop=True)
df_1

,image,class,bbox
0,1675,17,"[0.0, 14.01, 640.0, 293.56]"
1,1675,76,"[61.48, 367.82, 508.05, 105.71]"
2,4795,73,"[0.0, 192.0, 174.74, 282.61]"
3,4795,17,"[152.77, 113.55, 395.36, 364.39]"
4,4795,72,"[19.95, 2.49, 620.05, 349.09]"
...,...,...,...
1754,574810,17,"[64.52, 93.55, 215.05, 366.67]"
1755,575357,18,"[250.5, 158.39, 366.49, 199.85]"
1756,575357,34,"[39.52, 129.49, 102.3, 59.73]"
1757,579321,1,"[94.78, 0.95, 240.66, 211.01]"


In [132]:
real_file_list = []
real_imageid_list = []
for i in json_data['annotations']:
    if i['category_id'] == 17 or i['category_id'] == 18:
        real_file_list.append(f'{i["image_id"]}.jpg')
        real_imageid_list.append(i["image_id"])

In [ ]:
# 필요한 파일 말고 삭제
for i in os.listdir('val2017/val2017'):
    if i not in real_file_list:
        os.remove(f'val2017/val2017/{i}')

In [110]:
#파일이름 맞추기
for i in os.listdir('val2017/val2017'):
    os.rename(f'val2017/val2017/{i}', f'val2017/val2017/{i.lstrip("0")}')

In [125]:
len(os.listdir('val2017/val2017'))

349

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt') 
results = model.train(data='data2.yaml',epochs=10)
result = model('test111.png')

In [ ]:
### 혹시 yaml 파일 내용 보는 법(None이 있거나 할때 보려고 )
import yaml

yaml_file_path = 'data2.yaml'  # YAML 파일의 경로
with open(yaml_file_path, 'r') as file:
    data = yaml.safe_load(file)

print(data)  